In [1]:
import cv2

In [2]:
def output_keypoints(frame, net, threshold, BODY_PARTS, now_frame, total_frame):
    global points

    # 입력 이미지의 사이즈 정의
    image_height = 480
    image_width = 854

    # 네트워크에 넣기 위한 전처리
    input_blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (image_width, image_height), (0, 0, 0), swapRB=False, crop=False)

    # 전처리된 blob 네트워크에 입력
    net.setInput(input_blob)

    # 결과 받아오기
    out = net.forward()
    # The output is a 4D matrix :
    # The first dimension being the image ID ( in case you pass more than one image to the network ).
    # The second dimension indicates the index of a keypoint.
    # The model produces Confidence Maps and Part Affinity maps which are all concatenated.
    # For COCO model it consists of 57 parts – 18 keypoint confidence Maps + 1 background + 19*2 Part Affinity Maps. Similarly, for MPI, it produces 44 points.
    # We will be using only the first few points which correspond to Keypoints.
    # The third dimension is the height of the output map.
    out_height = out.shape[2]
    # The fourth dimension is the width of the output map.
    out_width = out.shape[3]

    # 원본 이미지의 높이, 너비를 받아오기
    frame_height, frame_width = frame.shape[:2]

    # 포인트 리스트 초기화
    points = []

    print(f"============================== frame: {now_frame:.0f} / {total_frame:.0f} ==============================")
    for i in range(len(BODY_PARTS)):

        # 신체 부위의 confidence map
        prob_map = out[0, i, :, :]

        # 최소값, 최대값, 최소값 위치, 최대값 위치
        min_val, prob, min_loc, point = cv2.minMaxLoc(prob_map)

        # 원본 이미지에 맞게 포인트 위치 조정
        x = (frame_width * point[0]) / out_width
        x = int(x)
        y = (frame_height * point[1]) / out_height
        y = int(y)

        if prob > threshold:  # [pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 1, lineType=cv2.LINE_AA)

            points.append((x, y))
            print(f"[pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

        else:  # [not pointed]
            cv2.circle(frame, (x, y), 5, (0, 255, 255), thickness=-1, lineType=cv2.FILLED)
            cv2.putText(frame, str(i), (x, y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 0), 1, lineType=cv2.LINE_AA)

            points.append(None)
            print(f"[not pointed] {BODY_PARTS[i]} ({i}) => prob: {prob:.5f} / x: {x} / y: {y}")

    return frame

In [3]:
def output_keypoints_with_lines(frame, POSE_PAIRS):
    for pair in POSE_PAIRS:
        part_a = pair[0]  # 0 (Head)
        part_b = pair[1]  # 1 (Neck)
        if points[part_a] and points[part_b]:
            print(f"[linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")
            cv2.line(frame, points[part_a], points[part_b], (0, 255, 0), 3)
        else:
            print(f"[not linked] {part_a} {points[part_a]} <=> {part_b} {points[part_b]}")

    return frame

In [4]:
def output_keypoints_with_lines_video(proto_file, weights_file, video_path, threshold, BODY_PARTS, POSE_PAIRS):

    # 네트워크 불러오기
    net = cv2.dnn.readNetFromCaffe(proto_file, weights_file)
    
    # GPU 사용
    # net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
    # net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    # 비디오 읽어오기
    capture = cv2.VideoCapture(video_path)

    while True:
        now_frame_boy = capture.get(cv2.CAP_PROP_POS_FRAMES)
        total_frame_boy = capture.get(cv2.CAP_PROP_FRAME_COUNT)

        if now_frame_boy == total_frame_boy:
            break

        ret, frame_boy = capture.read()
        frame_boy = output_keypoints(frame=frame_boy, net=net, threshold=threshold, BODY_PARTS=BODY_PARTS, now_frame=now_frame_boy, total_frame=total_frame_boy)
        frame_boy = output_keypoints_with_lines(frame=frame_boy, POSE_PAIRS=POSE_PAIRS)
        cv2.imshow("Output_Keypoints", frame_boy)

        if cv2.waitKey(10) == 27:  # esc 입력시 종료
            break

    capture.release()
    cv2.destroyAllWindows()

In [ ]:
BODY_PARTS_MPI = {0: "Head", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                  5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                  10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "Chest",
                  15: "Background"}

POSE_PAIRS_MPI = [[0, 1], [1, 2], [1, 5], [1, 14], [2, 3], [3, 4], [5, 6],
                  [6, 7], [8, 9], [9, 10], [11, 12], [12, 13], [14, 8], [14, 11]]

BODY_PARTS_COCO = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                   5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "RHip", 9: "RKnee",
                   10: "RAnkle", 11: "LHip", 12: "LKnee", 13: "LAnkle", 14: "REye",
                   15: "LEye", 16: "REar", 17: "LEar", 18: "Background"}

POSE_PAIRS_COCO = [[0, 1], [0, 14], [0, 15], [1, 2], [1, 5], [1, 8], [1, 11], [2, 3], [3, 4],
                   [5, 6], [6, 7], [8, 9], [9, 10], [12, 13], [11, 12], [14, 16], [15, 17]]

BODY_PARTS_BODY_25 = {0: "Nose", 1: "Neck", 2: "RShoulder", 3: "RElbow", 4: "RWrist",
                      5: "LShoulder", 6: "LElbow", 7: "LWrist", 8: "MidHip", 9: "RHip",
                      10: "RKnee", 11: "RAnkle", 12: "LHip", 13: "LKnee", 14: "LAnkle",
                      15: "REye", 16: "LEye", 17: "REar", 18: "LEar", 19: "LBigToe",
                      20: "LSmallToe", 21: "LHeel", 22: "RBigToe", 23: "RSmallToe", 24: "RHeel", 25: "Background"}

POSE_PAIRS_BODY_25 = [[0, 1], [0, 15], [0, 16], [1, 2], [1, 5], [1, 8], [8, 9], [8, 12], [9, 10], [12, 13], [2, 3],
                      [3, 4], [5, 6], [6, 7], [10, 11], [13, 14], [15, 17], [16, 18], [14, 21], [19, 21], [20, 21],
                      [11, 24], [22, 24], [23, 24]]

# 신경 네트워크의 구조를 지정하는 prototxt 파일 (다양한 계층이 배열되는 방법 등)
#protoFile_mpi = "pose/mpi/pose_deploy_linevec.prototxt"
#protoFile_mpi_faster = "pose/mpi/pose_deploy_linevec_faster_4_stages.prototxt"
#protoFile_coco = "pose/coco/pose_deploy_linevec.prototxt"
protoFile_body_25 = "body25/pose_deploy.prototxt"

# 훈련된 모델의 weight 를 저장하는 caffemodel 파일
#weightsFile_mpi = "pose/mpi/pose_iter_160000.caffemodel"
#weightsFile_coco = "pose/coco/pose_iter_440000.caffemodel"
weightsFile_body_25 = "body25/pose_iter_584000.caffemodel"


# 비디오 경로
path = "walk_side.mp4"

# 키포인트를 저장할 빈 리스트
points = []

#output_keypoints_with_lines_video(proto_file=protoFile_mpi_faster, weights_file=weightsFile_mpi, video_path=man,threshold=0.1, BODY_PARTS=BODY_PARTS_MPI, POSE_PAIRS=POSE_PAIRS_MPI)

#output_keypoints_with_lines_video(proto_file=protoFile_coco, weights_file=weightsFile_coco, video_path=man,threshold=0.1, BODY_PARTS=BODY_PARTS_COCO, POSE_PAIRS=POSE_PAIRS_COCO)

output_keypoints_with_lines_video(proto_file=protoFile_body_25, weights_file=weightsFile_body_25, video_path=path,
                                  threshold=0.1, BODY_PARTS=BODY_PARTS_BODY_25, POSE_PAIRS=POSE_PAIRS_BODY_25)

============================== frame: 0 / 139 ==============================
[pointed] Nose (0) => prob: 0.70836 / x: 157 / y: 32
[pointed] Neck (1) => prob: 0.35074 / x: 116 / y: 96
[pointed] RShoulder (2) => prob: 0.33997 / x: 134 / y: 104
[pointed] RElbow (3) => prob: 0.16255 / x: 130 / y: 224
[not pointed] RWrist (4) => prob: 0.06379 / x: 143 / y: 288
[pointed] LShoulder (5) => prob: 0.16386 / x: 98 / y: 88
[pointed] LElbow (6) => prob: 0.28199 / x: 152 / y: 176
[pointed] LWrist (7) => prob: 0.43227 / x: 188 / y: 200
[not pointed] MidHip (8) => prob: 0.05615 / x: 116 / y: 272
[not pointed] RHip (9) => prob: 0.06249 / x: 125 / y: 272
[not pointed] RKnee (10) => prob: 0.06323 / x: 134 / y: 344
[not pointed] RAnkle (11) => prob: 0.03347 / x: 121 / y: 400
[not pointed] LHip (12) => prob: 0.04063 / x: 103 / y: 264
[not pointed] LKnee (13) => prob: 0.04590 / x: 76 / y: 256
[not pointed] LAnkle (14) => prob: 0.03354 / x: 76 / y: 256
[pointed] REye (15) => prob: 0.76875 / x: 152 / y: 24
[n

============================== frame: 4 / 139 ==============================
[pointed] Nose (0) => prob: 0.73689 / x: 170 / y: 40
[pointed] Neck (1) => prob: 0.30097 / x: 130 / y: 96
[pointed] RShoulder (2) => prob: 0.25526 / x: 130 / y: 96
[pointed] RElbow (3) => prob: 0.11055 / x: 112 / y: 256
[not pointed] RWrist (4) => prob: 0.08025 / x: 112 / y: 248
[pointed] LShoulder (5) => prob: 0.15173 / x: 121 / y: 88
[not pointed] LElbow (6) => prob: 0.06969 / x: 112 / y: 256
[not pointed] LWrist (7) => prob: 0.02020 / x: 358 / y: 192
[not pointed] MidHip (8) => prob: 0.04740 / x: 130 / y: 288
[not pointed] RHip (9) => prob: 0.05284 / x: 125 / y: 296
[not pointed] RKnee (10) => prob: 0.01278 / x: 354 / y: 328
[not pointed] RAnkle (11) => prob: 0.01659 / x: 349 / y: 248
[not pointed] LHip (12) => prob: 0.02843 / x: 152 / y: 320
[not pointed] LKnee (13) => prob: 0.01527 / x: 358 / y: 328
[not pointed] LAnkle (14) => prob: 0.01789 / x: 349 / y: 248
[pointed] REye (15) => prob: 0.77265 / x: 165 

============================== frame: 8 / 139 ==============================
[pointed] Nose (0) => prob: 0.74493 / x: 179 / y: 40
[pointed] Neck (1) => prob: 0.25667 / x: 139 / y: 88
[pointed] RShoulder (2) => prob: 0.41187 / x: 139 / y: 96
[pointed] RElbow (3) => prob: 0.69346 / x: 143 / y: 168
[pointed] RWrist (4) => prob: 0.18047 / x: 201 / y: 224
[pointed] LShoulder (5) => prob: 0.14098 / x: 125 / y: 88
[pointed] LElbow (6) => prob: 0.30255 / x: 85 / y: 144
[pointed] LWrist (7) => prob: 0.23767 / x: 62 / y: 216
[pointed] MidHip (8) => prob: 0.11694 / x: 116 / y: 240
[pointed] RHip (9) => prob: 0.13034 / x: 112 / y: 240
[not pointed] RKnee (10) => prob: 0.04841 / x: 98 / y: 312
[not pointed] RAnkle (11) => prob: 0.01663 / x: 349 / y: 248
[not pointed] LHip (12) => prob: 0.06677 / x: 98 / y: 216
[not pointed] LKnee (13) => prob: 0.01840 / x: 112 / y: 320
[not pointed] LAnkle (14) => prob: 0.01793 / x: 349 / y: 248
[pointed] REye (15) => prob: 0.81256 / x: 174 / y: 32
[not pointed] LE

============================== frame: 12 / 139 ==============================
[pointed] Nose (0) => prob: 0.79430 / x: 192 / y: 56
[pointed] Neck (1) => prob: 0.23268 / x: 152 / y: 96
[pointed] RShoulder (2) => prob: 0.36576 / x: 148 / y: 96
[pointed] RElbow (3) => prob: 0.32146 / x: 179 / y: 168
[pointed] RWrist (4) => prob: 0.24879 / x: 40 / y: 208
[pointed] LShoulder (5) => prob: 0.17828 / x: 130 / y: 96
[pointed] LElbow (6) => prob: 0.35760 / x: 80 / y: 160
[pointed] LWrist (7) => prob: 0.40578 / x: 44 / y: 208
[not pointed] MidHip (8) => prob: 0.07398 / x: 130 / y: 256
[not pointed] RHip (9) => prob: 0.06688 / x: 134 / y: 264
[not pointed] RKnee (10) => prob: 0.06035 / x: 192 / y: 320
[not pointed] RAnkle (11) => prob: 0.02210 / x: 188 / y: 392
[not pointed] LHip (12) => prob: 0.04090 / x: 148 / y: 256
[not pointed] LKnee (13) => prob: 0.02868 / x: 192 / y: 312
[not pointed] LAnkle (14) => prob: 0.01890 / x: 309 / y: 264
[pointed] REye (15) => prob: 0.83918 / x: 188 / y: 48
[not p

============================== frame: 16 / 139 ==============================
[pointed] Nose (0) => prob: 0.69356 / x: 201 / y: 48
[pointed] Neck (1) => prob: 0.31835 / x: 161 / y: 96
[pointed] RShoulder (2) => prob: 0.33187 / x: 165 / y: 104
[pointed] RElbow (3) => prob: 0.16687 / x: 183 / y: 216
[not pointed] RWrist (4) => prob: 0.06842 / x: 210 / y: 272
[pointed] LShoulder (5) => prob: 0.18784 / x: 157 / y: 96
[not pointed] LElbow (6) => prob: 0.02545 / x: 161 / y: 168
[not pointed] LWrist (7) => prob: 0.02029 / x: 358 / y: 192
[not pointed] MidHip (8) => prob: 0.09870 / x: 134 / y: 232
[not pointed] RHip (9) => prob: 0.09661 / x: 148 / y: 248
[not pointed] RKnee (10) => prob: 0.04205 / x: 143 / y: 344
[not pointed] RAnkle (11) => prob: 0.01682 / x: 349 / y: 248
[not pointed] LHip (12) => prob: 0.08442 / x: 125 / y: 224
[not pointed] LKnee (13) => prob: 0.02968 / x: 134 / y: 344
[not pointed] LAnkle (14) => prob: 0.01815 / x: 354 / y: 248
[pointed] REye (15) => prob: 0.72438 / x: 19

============================== frame: 20 / 139 ==============================
[pointed] Nose (0) => prob: 0.72724 / x: 210 / y: 40
[pointed] Neck (1) => prob: 0.30302 / x: 165 / y: 88
[pointed] RShoulder (2) => prob: 0.27073 / x: 165 / y: 96
[pointed] RElbow (3) => prob: 0.13656 / x: 125 / y: 152
[not pointed] RWrist (4) => prob: 0.05404 / x: 112 / y: 240
[pointed] LShoulder (5) => prob: 0.13481 / x: 161 / y: 80
[not pointed] LElbow (6) => prob: 0.09494 / x: 179 / y: 176
[pointed] LWrist (7) => prob: 0.11537 / x: 224 / y: 216
[not pointed] MidHip (8) => prob: 0.05373 / x: 143 / y: 256
[not pointed] RHip (9) => prob: 0.06164 / x: 139 / y: 256
[not pointed] RKnee (10) => prob: 0.02560 / x: 121 / y: 416
[not pointed] RAnkle (11) => prob: 0.01680 / x: 349 / y: 248
[not pointed] LHip (12) => prob: 0.03570 / x: 107 / y: 296
[not pointed] LKnee (13) => prob: 0.02022 / x: 116 / y: 248
[not pointed] LAnkle (14) => prob: 0.01813 / x: 354 / y: 248
[pointed] REye (15) => prob: 0.77654 / x: 206 / y

In [ ]:
points